In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [3]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

100% 172M/172M [00:06<00:00, 36.7MB/s]
100% 172M/172M [00:06<00:00, 29.7MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [6]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

In [7]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [8]:
input = (256, 256, 3)

In [9]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [10]:
# base model
base_model = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=input
)
for layer in base_model.layers:
    layer.trainable = False

29084464/29084464 [==============================] - 2s 0us/step


In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [12]:
model = Model(inputs=base_model.input, outputs=predictions)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 262, 262, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 128, 128, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1/conv[0][0]']      

In [14]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [15]:
history = model.fit(x_train,epochs=10, batch_size=100, validation_data=x_test)

Epoch 1/10
89/89 [==============================] - 64s 461ms/step - loss: 0.5381 - accuracy: 0.7206 - val_loss: 0.5258 - val_accuracy: 0.6933
Epoch 2/10
89/89 [==============================] - 37s 403ms/step - loss: 0.2983 - accuracy: 0.8798 - val_loss: 0.5772 - val_accuracy: 0.7017
Epoch 3/10
89/89 [==============================] - 37s 408ms/step - loss: 0.1733 - accuracy: 0.9392 - val_loss: 0.7556 - val_accuracy: 0.6767
Epoch 4/10
89/89 [==============================] - 38s 409ms/step - loss: 0.1073 - accuracy: 0.9645 - val_loss: 0.9820 - val_accuracy: 0.6450
Epoch 5/10
89/89 [==============================] - 36s 390ms/step - loss: 0.0737 - accuracy: 0.9786 - val_loss: 0.9506 - val_accuracy: 0.6617
Epoch 6/10
89/89 [==============================] - 38s 414ms/step - loss: 0.0499 - accuracy: 0.9866 - val_loss: 0.9071 - val_accuracy: 0.6833
Epoch 7/10
89/89 [==============================] - 38s 418ms/step - loss: 0.0414 - accuracy: 0.9879 - val_loss: 0.9115 - val_accuracy: 0.6833